In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/1.year.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(f'''https://www.auveco.com/store_ss2/extensions/UnlockCommerce/YearMakeModel/1.0.0/Modules/YearMakeModel/SuiteScript2/YearMakeModel.Service.ss?action=getmakes&year={input_.loc[a, 'Year Code']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_make_code = [dict_['makeId'] if dict_['makeId'] else '' for dict_ in json_]
                        while '' in list_make_code:
                            list_make_code.remove('')
                        list_make = [dict_['make'] if dict_['make'] else '' for dict_ in json_]
                        while '' in list_make:
                            list_make.remove('')
                        if list_make:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make': list_make,
                                    'Make Code': list_make_code,
                                    'Year': input_.loc[a, 'Year'],
                                    'Year Code': input_.loc[a, 'Year Code']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Year'],
                                                ascending=[True, False],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/2.make-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Year'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/make_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：69

[状态：ok，尝试次数：4] - 2024
[剩余数量：39] - [当前时间：15:49:30]

[状态：ok，尝试次数：1] - 2022
[剩余数量：38] - [当前时间：15:49:31]

[状态：ok，尝试次数：7] - 1998
[剩余数量：37] - [当前时间：15:49:31]

[状态：ok，尝试次数：7] - 2001
[剩余数量：36] - [当前时间：15:49:32]

[状态：ok，尝试次数：18] - 2013
[剩余数量：35] - [当前时间：15:49:33]

[状态：ok，尝试次数：34] - 1999
[剩余数量：34] - [当前时间：15:49:33]

[状态：ok，尝试次数：39] - 2015
[剩余数量：33] - [当前时间：15:49:34]

[状态：ok，尝试次数：30] - 2017
[剩余数量：32] - [当前时间：15:49:34]

[状态：ok，尝试次数：55] - 2008
[剩余数量：31] - [当前时间：15:49:35]

[状态：ok，尝试次数：56] - 1995
[剩余数量：30] - [当前时间：15:49:36]

[状态：ok，尝试次数：7] - 1990
[剩余数量：29] - [当前时间：15:49:36]

[状态：ok，尝试次数：38] - 1992
[剩余数量：28] - [当前时间：15:49:36]

[状态：ok，尝试次数：24] - 1991
[剩余数量：27] - [当前时间：15:49:36]

[状态：ok，尝试次数：19] - 1994
[剩余数量：26] - [当前时间：15:49:36]

[状态：ok，尝试次数：42] - 1997
[剩余数量：25] - [当前时间：15:49:36]

[状态：ok，尝试次数：36] - 1996
[剩余数量：24] - [当前时间：15:49:36]

[状态：ok，尝试次数：8] - 1989
[剩余数量：23] - [当前时间：15:49:36]

[状态：ok，尝试次数：50] - 2018
[剩余数量：22] - [当前时间：15:49:37]

[状态：ok，尝试次数：64] - 2012
[剩余数量：21] - [当前时间：15:49:37]

[状态：ok，尝试次